In [ ]:
import matplotlib.pyplot as plt
import hddm
from IPython.display import clear_output

In [2]:
import pandas as pd

In [13]:
include = {'a', 't', 'v'}
v = [0, 0.05, 0.1]
cond_params, merged_params = hddm.generate.gen_rand_params(
            include=include, cond_dict={"v": v}
        )

In [15]:
true_params = {'a': cond_params['c0']['a'],
 'v(c0)': cond_params['c0']['v'],
 'v(c1)': cond_params['c1']['v'],
 't': cond_params['c0']['t'],
              'z': cond_params['c0']['z']}

In [98]:
n_samples = 28
n_subj = 140
n_sim = 100

In [ ]:
m_list = []
true_params_list = []

for k in range(n_sim):
    print(k)
    samples_list = []
    true_params = []

    for i in range(n_subj):
        v = [0, 0.05, 0.1]
        include = {'a', 't', 'v'}
        cond_params, merged_params = hddm.generate.gen_rand_params(
                    include=include, cond_dict={"v": v}
                )

        true_params.append(merged_params)

        samples, _ = hddm.generate.gen_rand_data(cond_params, size=n_samples)
        samples = hddm.utils.flip_errors(samples)
        samples["subj_idx"] = i
        samples_list.append(samples)
        
    sim_df = pd.concat(samples_list)
    sim_df.to_csv("power_sim_data/data/power_sim_data_{}.csv".format(k))
    true_params_list.append(true_params)

In [141]:
for i in range(n_sim):
    pd.DataFrame(true_params_list[i]).to_csv("power_sim_data/true_parameters/true_parameters_{}.csv".format(i))

In [142]:
from multiprocessing import Pool

In [143]:
import logging
logger = logging.getLogger('hddm')
logger.setLevel(logging.ERROR)

In [144]:
def sim_fit(i):
    sim_df = pd.read_csv("power_sim_data/data/power_sim_data_{}.csv".format(i))
    
    m = hddm.HDDM(sim_df, depends_on={'v': 'condition'}, p_outlier=.05)
    
    m.find_starting_values()
    m.sample(1000, burn=500, dbname='power_sim_data/models/traces_hddm_power_{}.db'.format(i), db='pickle')
    m.save("power_sim_data/models/power_model_sim_{}".format(i))
    
    return m

In [ ]:
with Pool(processes=12) as P:
    power_sims = P.map(sim_fit, range(100))

In [178]:
sim_result = []
for i in range(len(power_sims)):

    m = power_sims[i]
    v_c0, v_c1, v_c2 = m.nodes_db.loc[["v(c0)",
                                   "v(c1)",
                                   "v(c2)"], 'node']
    
    
    sim_result.append({"vc1 > 0":(v_c1.trace() > 0).mean(),
    "vc2 > 0": (v_c2.trace() > 0).mean(),
    "vc1 > vc0": (v_c1.trace() > v_c0.trace()).mean(),
    "vc2 > vc1": (v_c2.trace() > v_c1.trace()).mean(),
    "vc2 > vc0": (v_c2.trace() > v_c0.trace()).mean()})

In [180]:
sim_result_df = pd.DataFrame(sim_result)

In [188]:
(sim_result_df["vc1 > 0"] > 0.975).mean()

0.49

In [189]:
(sim_result_df["vc2 > 0"] > 0.975).mean()

0.94

In [190]:
(sim_result_df["vc1 > vc0"] > 0.975).mean()

0.4

In [176]:
v_c0, v_c1, v_c2 = power_sims[1].nodes_db.loc[["v(c0)",
                                   "v(c1)",
                                   "v(c2)"], 'node']

In [ ]:
hddm.analyze.plot_posterior_nodes([v_c0, v_c1, v_c2])